## 收集正在營業店家的url

In [ ]:
import random
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import datetime
import os
import csv


options = uc.ChromeOptions()
options.headless=True
options.add_argument('--headless')
browser = uc.Chrome(options=options)

#chrome_options = Options()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
#browser = webdriver.Chrome(executable_path='../DAY18/chromedriver',chrome_options=chrome_options)

url = "https://www.foodpanda.com.tw/en/city/taipei-city"
browser.get(url)
#輸入區域
browser.find_element(By.ID,"delivery-information-postal-index").send_keys("信義區忠孝東路")
#點擊搜尋
browser.find_element(By.XPATH,'//*[@id="seo-city-page-root"]/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/button').click()


time.sleep(10)
SCROLL_PAUSE_TIME = 5
SCROLL_PAUSE_TIME_1 = 5

last_height = ''
while True:
    delay_choices = [5, 6, 7, 8, 9]  #延遲的秒數
    delay = random.choice(delay_choices)  #隨機選取秒數   
    time.sleep(delay)
    browser.execute_script("window.scrollBy(0, 1000);")
    time.sleep(delay)
    browser.execute_script("window.scrollBy(0, 600);")
    #browser.execute_script("window.scrollBy(0, document.body.scrollHeight);")
    time.sleep(delay)
    browser.execute_script("window.scrollBy(0, -500);")
    
    new_height = browser.execute_script("return document.body.scrollHeight")
    my_current_url = browser.current_url
    print(my_current_url)
    print(browser.execute_script("return document.body.scrollHeight"))
    try:
        browser.find_element(By.XPATH, '//*[@id="seo-city-page-root"]/div/div[3]/div[1]/section[2]/span')
        print('Closed for now')
        break
    except:
        print('網頁更新中...')


html_source = browser.page_source
browser.quit()
soup = BeautifulSoup(html_source, "html.parser")
total_info = soup.find_all("section",class_="vendor-list-section")


data_profile = {
'name' : '',
'score' : '',
'comment_cnt' : '',
'style' : ''
}

data_url = {
    'name' : '',
    'url' : ''
}

data_feature = {
    'name' : '',
    'feature' : ''
}


createtime = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

shop_profile_path = f'./shop_profile_{createtime}.csv'
shop_url_path = f'./shop_url_{createtime}.csv'
shop_feature_path = f'./shop_feature_{createtime}.csv'


with open(shop_profile_path,'w',newline='', encoding="utf8") as shop_profile_file,\
     open(shop_url_path,'w',newline='', encoding="utf8") as shop_url_file,\
     open(shop_feature_path,'w',newline='', encoding="utf8") as shop_feature_file:
    
    shop_profile_writer = csv.DictWriter(shop_profile_file, fieldnames=[*data_profile.keys()], delimiter=",")
    shop_url_writer = csv.DictWriter(shop_url_file, fieldnames=[*data_url.keys()], delimiter=",")
    shop_feature_writer = csv.DictWriter(shop_feature_file, fieldnames=[*data_feature.keys()], delimiter=",")

    shop_profile_writer.writeheader()
    shop_url_writer.writeheader()
    shop_feature_writer.writeheader()

    # 正在營業的店
    #print('opening')
    open_info = total_info[0]

    for x in open_info.find_all("li",class_="vendor-tile-wrapper"):
        # 店名
        name = x.find("a",class_="name fn").text
        #print(name)
        data_profile["name"] = name
        data_url['name'] = name
        
        # 評價分數
        try:
            score = x.find("span",class_="rating--label-primary cl-rating-tag-text f-label-small-font-size fw-label-small-font-weight lh-label-small-line-height").text.split('/')[0]
            #print(score)
            data_profile["score"] = score
        except:
            #print('0')
            data_profile["score"] = '0'
    
        # 評論數
        try:
            comment_cnt = x.find("span",class_="rating--label-secondary cl-neutral-secondary f-label-small-secondary-font-size fw-label-small-secondary-font-weight lh-label-small-secondary-line-height").text\
                         .replace('(','')\
                         .replace(')','')\
                         .replace('+','')
            #print(comment_cnt)
            data_profile["comment_cnt"] = comment_cnt
        except:
            #print('0')
            data_profile["comment_cnt"] = '0'
    
        # 店家類型
        try:
            tag = x.find("li",class_="vendor-characteristic").text
            #print(tag)
            data_profile["style"] = tag
        except:
            #print('None')
            data_profile["style"] = 'None'        
    
        #網址
        url = 'https://www.foodpanda.com.tw/' + x.find("a",class_="name fn").get('href')
        #print(url)
        data_url["url"] = url
    
        # 特徵、優惠
        try:
            features = x.find_all("span",class_="box-flex fd-row ai-center")
            # 定位的到，但回傳是空list
            if len(features) == 0:
                #print('None')
                data_feature["name"] = name
                data_feature["feature"] = 'None'
                shop_feature_writer.writerow(data_feature)
            else:
                for j in features:
                    feature = j.text
                    #print(feature)
                    data_feature["name"] = name
                    data_feature["feature"] = 'None'
                    shop_feature_writer.writerow(data_feature)
        except:
            #print('None')
            data_feature["name"] = name
            data_feature["feature"] = 'None'
            shop_feature_writer.writerow(data_feature)
        
        shop_profile_writer.writerow(data_profile)
        shop_url_writer.writerow(data_url)
        #print('--------------------------------------------------------------------------------------------------')
    # 尚未營業的店
    # close_info = total_info[1]

### 測試